# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    
        num. electrons       : 12
        spin polarization    : none
        temperature          : 0.001 Ha
        smearing             : DFTK.Smearing.FermiDi

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    
        num. electrons       : 60
        spin polarization    : none
        temperature          : 0.001 Ha
        smearing             : DFTK.Smearing.FermiD

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355220941914                   -1.10    6.0    178ms
  2   -9.356779167978       -2.81       -1.43    1.0   80.4ms
  3   -9.357055445704       -3.56       -2.77    2.4   97.9ms
  4   -9.357110688892       -4.26       -3.02    8.1    214ms
  5   -9.357111025956       -6.47       -3.13    2.1   93.8ms
  6   -9.357111257288       -6.64       -3.28    1.0   81.1ms
  7   -9.357111403739       -6.83       -3.44    1.0   79.9ms
  8   -9.357111462592       -7.23       -3.59    1.0    121ms
  9   -9.357111497987       -7.45       -3.87    1.9    588ms
 10   -9.357111507101       -8.04       -4.17    1.4   87.3ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.75229562721                   -0.97    7.1    405ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     3
│     1
│     1
│     6
│     1
│     3
│     1
│     5
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  2   -18.78900242509       -1.44       -1.33    2.6    265ms
  3   -18.79245089160       -2.46       -2.38    8.4    396ms
  4   -18.79260053755       -3.82       -2.84    4.8    386ms
  5   -18.79260572315       -5.29       -3.15    2.5    268ms
  6   -18.79260623472       -6.29       -3.41    3.5    274ms
  7   -18.79260636662       -6.88       -3.98    2.0    234ms
  8   -18.79260639197       -7.60       -4.35    3.8    316ms


In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.55069416918                   -0.84   10.6    2.35s
  2   -37.55785987563       -2.14       -1.22    2.2    789ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     13
│      6
│     14
│     19
│     24
│     19
│     14
│      9
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  3   -37.56399378544       -2.21       -2.12   14.8    1.59s
  4   -37.56487935335       -3.05       -2.26    9.5    1.51s
  5   -37.56494357218       -4.19       -2.72    2.0    840ms
  6   -37.56494093148   +   -5.58       -2.92    6.8    1.15s
  7   -37.56494732435       -5.19       -3.18    1.4    1.30s
  8   -37.56494970880       -5.62       -3.92    3.5    1.01s
  9   -37.56494985991       -6.82       -4.14    4.6    1.46s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355306942717                   -1.10    6.4    212ms
  2   -9.356840733573       -2.81       -1.91    1.0   87.7ms
  3   -9.357095141836       -3.59       -2.81    4.8    148ms
  4   -9.357093150352   +   -5.70       -2.63    4.9    168ms
  5   -9.357111127110       -4.75       -3.48    1.1   90.4ms
  6   -9.357111437590       -6.51       -3.90    2.5    117ms
  7   -9.357111504196       -7.18       -4.44    3.5    116ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     11
│      2
│     16
│      3
│     14
│      9
│     15
│      2
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54990867446                   -0.84    9.0    2.33s
  2   -37.55559163375       -2.25       -1.61    2.4    763ms
┌ Warning: Detected large minimal occupation 0.23432495495993672. SCF could be unstable. Try switching to adaptive band selection (`nbandsalg=AdaptiveBands(model)`) or request more converged bands than 29 (e.g. `nbandsalg=AdaptiveBands(model; n_bands_converge=32`)
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:87
  3   -33.53397120561   +    0.60       -0.77    9.6    1.51s
  4   -37.44340209627        0.59       -1.51    7.6    1.49s
  5   -37.53309911462       -1.05       -1.79    1.9    774ms
  6   -37.5292

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.